In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

Using TensorFlow backend.


In [2]:
from notebook.services.config import ConfigManager
ConfigManager().update('notebook', {'ExecuteTime': {
   	'display_absolute_timestamps': False,
    'relative_timing_update_period': 5,
    'template': {
    	'executed': 'started ${start_time}, finished in ${duration}',
    }
}})

{'Cell': {'cm_config': {'lineNumbers': True}},
 'ExecuteTime': {'display_absolute_timestamps': False,
  'relative_timing_update_period': 5,
  'template': {'executed': 'started ${start_time}, finished in ${duration}'}}}

## Finding composition with Enrichment and Burnup
Testing time for various methods

In [3]:
sample_num = 100
enr = np.random.uniform(low=3, high=5, size=(sample_num,))
br = np.random.uniform(low=30000, high=50000, size=(sample_num,))
enr_br = np.column_stack((enr,br))
print(np.shape(enr_br))
enr_br

(100, 2)


array([[  3.30125368e+00,   3.80530482e+04],
       [  4.05540160e+00,   3.85476153e+04],
       [  3.02528571e+00,   3.75077647e+04],
       [  4.88270805e+00,   3.43296561e+04],
       [  3.12896480e+00,   4.91845324e+04],
       [  3.22103932e+00,   4.84838728e+04],
       [  4.77048197e+00,   3.77587663e+04],
       [  3.85732648e+00,   4.74552984e+04],
       [  4.25847007e+00,   4.28728501e+04],
       [  4.09670970e+00,   4.11647460e+04],
       [  3.79479937e+00,   4.41708617e+04],
       [  4.26860909e+00,   4.65425507e+04],
       [  4.07762779e+00,   3.51150313e+04],
       [  4.69787558e+00,   3.02441507e+04],
       [  3.83063256e+00,   4.15807268e+04],
       [  4.27736456e+00,   3.60368776e+04],
       [  3.06599153e+00,   3.20360743e+04],
       [  3.14734489e+00,   3.34071561e+04],
       [  4.47969400e+00,   3.53072550e+04],
       [  3.10702225e+00,   4.27563823e+04],
       [  4.69685117e+00,   3.98900654e+04],
       [  4.48230771e+00,   4.79645879e+04],
       [  

## Pickle and model

In [5]:
# load pickled dictionary with
# key: isotope
# value: trained model
model_dict = pickle.load(open('./script/ann_model.pkl',"rb"))

In [9]:
# get time for rom depletion for one assembly
# given enrichment and burnup value,
# predict UNF composition

def predict(enr_bu):
    model = model_dict['model']
    iso_list = model_dict['iso_list']
    x = model_dict['xscaler'].transform(enr_bu)
    y = model_dict['yscaler'].inverse_transform(model.predict(x))
    comp_dict = {}
    for indx, i in enumerate(y):
        comp_dict[indx] = {}
        for indx2, iso in enumerate(iso_list):
            comp_dict[indx][iso] = i[indx2]
    return comp_dict


%time predict(enr_br)


CPU times: user 24.5 ms, sys: 1.11 ms, total: 25.6 ms
Wall time: 11.5 ms


{0: {'zr-95': 0.00012185197,
  'zr-93': 0.08755032,
  'ra-226': 6.7134306e-12,
  'ru-103': 6.9605094e-06,
  'ru-106': 0.0084606903,
  'sb-124': 8.6076852e-08,
  'sb-125': 0.00096349022,
  'pu-238': 0.024905523,
  'pu-239': 0.70353645,
  'np-237': 0.055631686,
  'nb-95': 0.00014372851,
  'pa-231': 6.0816404e-08,
  'pd-107': 0.026917364,
  'pm-147': 0.014128407,
  'pu-240': 0.25771713,
  'pu-241': 0.16621225,
  'pu-244': 4.5713655e-06,
  'pu-242': 0.060031626,
  'i-129': 0.018128773,
  'h-3': 5.9071876e-06,
  'kr-85': 0.0027857188,
  'nb-94': 1.686565e-07,
  'cm-248': 4.6440583e-08,
  'cs-134': 0.010660433,
  'cs-135': 0.053492106,
  'cm-245': 0.00044399418,
  'cm-244': 0.0060288832,
  'cm-246': 3.947199e-05,
  'cm-247': 6.1085694e-07,
  'eu-152': 2.0034602e-06,
  'cs-137': 0.1319838,
  'eu-154': 0.0030650173,
  'eu-155': 0.00084532611,
  'am-241': 0.015744923,
  'am-242m': 0.00012324833,
  'am-243': 0.015807323,
  'cf-251': 1.1606183e-10,
  'cf-249': 4.3628692e-10,
  'ag-108m': 2.373706

## Looking up from db

In [10]:

df = pd.read_csv('./curated.csv', index_col=0)

In [11]:
# get time for `look-up' method for one assembly
# find closest assembly to this burnup and enrichment
def deplete_lookup(df, enr_br):
    depleted_composition_lookup = {}
    for indx, enr_br_set in enumerate(enr_br):
        df['err'] = abs(df['bu'] - enr_br[0][1]) + abs(df['init_enr'] - enr_br[0][0]) * 10000
        minimum_err = df['err'].argmin()
        df = df.drop(columns='err')
        average_assem = df.loc[minimum_err]
        depleted_composition_lookup[indx] = average_assem.iloc[5:].to_dict()
    return depleted_composition_lookup

%time deplete_lookup(df, enr_br)

/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  import sys


CPU times: user 7.5 s, sys: 14.2 s, total: 21.8 s
Wall time: 21.6 s


{0: {'zr-95': 0.00010368026681391106,
  'zr-93': 0.088346609720346581,
  'ra-226': 5.461694729582595e-12,
  'ru-103': 6.0978822532825071e-06,
  'ru-106': 0.0080243392297611061,
  'sb-124': 8.2527984193404625e-08,
  'sb-125': 0.00095900851755512247,
  'pu-238': 0.025227927202733782,
  'pu-239': 0.67147401824144981,
  'np-237': 0.05486363041660846,
  'nb-95': 0.00012225589254708386,
  'pa-231': 5.4830274242656103e-08,
  'pd-107': 0.028039396741552852,
  'pm-147': 0.013647926714875899,
  'pu-240': 0.26242110561901405,
  'pu-241': 0.16022123911467098,
  'pu-244': 4.9070575295320345e-06,
  'pu-242': 0.064899973611323644,
  'i-129': 0.018319713450020068,
  'h-3': 6.0093307255111985e-06,
  'kr-85': 0.0024620142764956729,
  'nb-94': 1.6761636851897433e-07,
  'cm-248': 4.8958190579464274e-08,
  'cs-134': 0.010699645637774395,
  'cs-135': 0.054528948821490585,
  'cm-245': 0.00048724216156522548,
  'cm-244': 0.0068035507183707329,
  'cm-246': 4.5340910653263893e-05,
  'cm-247': 6.792696989251605e